# Spectral extraction

- select the object(s) with lines in all grisms
- extract the three spectra with specreduce

**Author notes**:
- Combine all the exposures for a single target
- Find something more automatic on multiple spectra using specreduce?
- Extract with boxcar and optimal (also self profile)
- Still need to take care of NaNs in specreduce


In [ ]:
# utility
import os
import glob
import shutil
import numpy as np

# astropy
from astropy.table import Table, join
from astropy.io import fits
from astropy import units as u

# jdaviz
from jdaviz import Specviz2d, Specviz

# specutils
from specutils import Spectrum1D

# 
from matplotlib import pyplot as plt


In [ ]:
# Update appropriately
data_dir_images = "/Users/cpacifici/DATA/NGDEEP/image3/"
data_dir_out_spec2 = "/Users/cpacifici/DATA/NGDEEP/spec2/"

## Select files and interesting object

In [ ]:
calfile_f115w = data_dir_out_spec2 + 'jw02079004001_03101_00001_nis_cal.fits'
calfile_f150w = data_dir_out_spec2 + 'jw02079004002_09101_00001_nis_cal.fits'
calfile_f200w = data_dir_out_spec2 + 'jw02079004003_03101_00001_nis_cal.fits'
calfiles = [calfile_f115w, calfile_f150w, calfile_f200w]

In [ ]:
# My favorite object from the exploration in notebook 07
object_id = 109

### Find correct extension in the files

**Developer note:** there might be a way to know how many SCI extensions there are in a fits file, but I do not know how. For now I hard code `201` because this is the number of objects in the catalog that was used to generate the `cal` files.

In [ ]:
ext_num = []
for file in calfiles:
    with fits.open(file) as hdu:
        print(file)
        for ii in range(1, 202):
            if hdu['SCI',ii].header['SOURCEID']==object_id:
                ext_num.append(ii)
ext_num

## Load in Specviz2d and extract 1D spectrum

**Developer note**: the NaNs are not yet treated properly by Specviz2d (and specreduce). Need to mask them to zeros before proceeding.

### F115W

In [ ]:
filter_num = 0
calimage = fits.open(calfiles[filter_num])
data = calimage['SCI',ext_num[filter_num]].data
data = np.transpose(data)
data = np.nan_to_num(data)
wave = calimage['WAVELENGTH',ext_num[filter_num]].data[:,0]

In [ ]:
spec2d = Spectrum1D(flux=data*u.Jy, spectral_axis=wave*u.um)

In [ ]:
specviz2d = Specviz2d()
specviz2d.load_data(spec2d)
specviz2d.show()

In [ ]:
spec1d_f115w = specviz2d.get_data('Spectrum 1D')

### F150W

In [ ]:
filter_num = 1
calimage = fits.open(calfiles[filter_num])
data = calimage['SCI',ext_num[filter_num]].data
data = np.transpose(data)
data = np.nan_to_num(data)
wave = calimage['WAVELENGTH',ext_num[filter_num]].data[:,0]

In [ ]:
spec2d = Spectrum1D(flux=data*u.Jy, spectral_axis=wave*u.um)

In [ ]:
specviz2d2 = Specviz2d()
specviz2d2.load_data(spec2d)
specviz2d2.show()

In [ ]:
spec1d_f150w = specviz2d2.get_data('Spectrum 1D')

### F200W

In [ ]:
filter_num = 2
calimage = fits.open(calfiles[filter_num])
data = calimage['SCI',ext_num[filter_num]].data
data = np.transpose(data)
data = np.nan_to_num(data)
wave = calimage['WAVELENGTH',ext_num[filter_num]].data[:,0]

In [ ]:
spec2d = Spectrum1D(flux=data*u.Jy, spectral_axis=wave*u.um)

In [ ]:
specviz2d3 = Specviz2d()
specviz2d3.load_data(spec2d)
specviz2d3.show()

In [ ]:
spec1d_f200w = specviz2d3.get_data('Spectrum 1D')

## Open in Specviz

In [ ]:
specviz = Specviz()
specviz.load_data(spec1d_f115w, data_label='f115w')
specviz.load_data(spec1d_f150w, data_label='f150w')
specviz.load_data(spec1d_f200w, data_label='f200w')
specviz.show()

Load a line list and measure the redshift of the target (spoiler: z=2.23)

### Save spectra as fits files

In [ ]:
spec1d_f115w.write('source109_f115w.fits')
spec1d_f150w.write('source109_f150w.fits')
spec1d_f200w.write('source109_f200w.fits')